In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import torch 
import torch.nn as nn
from torch.utils.data import DataLoader

# Libraries for processing sounds
import librosa
from IPython.display import Audio
import random

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [19]:
df = pd.read_csv('data/Animal_Sound_processed.csv')
len(df)

650

In [20]:
from ClassesData.AnimalSoundDataset import AnimalSoundDataset

dataset_train = AnimalSoundDataset(df, split='train', split_ratio=0.8, seed=42)
dataset_val = AnimalSoundDataset(df, split='val', split_ratio=0.8, seed=42)

Classes found: ['bear', 'cat', 'chicken', 'cow', 'dog', 'dolphin', 'donkey', 'elephant', 'frog', 'horse', 'lion', 'monkey', 'sheep']
Total segments extracted: 2909
train set contains 2327 segments.
Classes found: ['bear', 'cat', 'chicken', 'cow', 'dog', 'dolphin', 'donkey', 'elephant', 'frog', 'horse', 'lion', 'monkey', 'sheep']
Total segments extracted: 2909
val set contains 582 segments.


In [21]:
# in a paper there are 20000 samples in the training set
# and they use 1000 for batch
# we have 2000 samples in the training set
# therefore we use 128 for batch
batch_size = 128

In [22]:
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=False)

print(f"Number of batches in train loader: {len(train_loader)}")

for batch in train_loader:
    x_batch, y_batch = batch
    print(x_batch.shape)  # [batch_size, 1, 128, 400]
    print(y_batch.shape)  # [batch_size]
    break

Number of batches in train loader: 19
torch.Size([128, 2, 60, 41])
torch.Size([128])


In [ ]:
input_dim = dataset_train[0][0].shape[0]  # e.g., (2, 60, 41) for short segments
n_classes = len(dataset_train.classes)
print(f"Input dimension: {input_dim}, Number of classes: {n_classes}")

hyperparameters = dict(
    input_dim=input_dim,
    output_dim=n_classes,
    hidden_layers_size=5000,
    activation='relu',
    kernel_size_conv=[(57, 6), (1, 3)],
    kernel_size_pool=[(4, 3), (1, 3)],
    stride_conv=[(1, 1), (1, 1)],
    stride_pool=[(1, 3), (1, 3)],
    filters=[80, 80],
    batch_normalization=False,
    dropout_rate=0.5,
    # trainer hyperparameters
    learning_rate=0.002,
    weight_decay=0.001,
    momentum=0.9,
    nesterov=True,

    # questionable hyperparameters
    #batch_size=batch_size,
    max_epoch=100,

    #Early stopping and sceduler
    patience_lr=5,
    early_stopping=True
)

Input dimension: 2, Number of classes: 13


In [19]:
from sklearn.model_selection import ParameterSampler
import pandas as pd
from ClassesML.AudioTrainer import AudioTrainer
from ClassesML.AudioModel import AudioModel

hyperparameter_choices = {}
for k in hyperparameters.keys():
    hyperparameter_choices[k] = [hyperparameters[k]]

hyperparameter_choices['learning_rate'] = [0.005, 0.001]
hyperparameter_choices['batch_size'] = [64, 128, 256]
hyperparameter_choices['max_epoch'] = [100]
hyperparameter_choices['hidden_layers_size']=[1000, 5000]
hyperparameter_choices['patience_lr'] = [5, 10, 15]
hyperparameter_choices['momentum'] = [0.9, 0.95, 0.85]
hyperparameter_choices['weight_decay'] = [0.001, 0.002]


hyperparameter_try = list(ParameterSampler(hyperparameter_choices, n_iter=20))

metric_list = []

for hyperparam in hyperparameter_try:

    model = AudioModel(hyperparam).to(device)
    
    train_loader = DataLoader(dataset_train, batch_size=hyperparam['batch_size'], shuffle=True)
    val_loader = DataLoader(dataset_val, batch_size=hyperparam['batch_size'], shuffle=False)

    trainer = AudioTrainer(model, train_loader, val_loader, hyperparam, device=device)

    train_accuracy_list, valid_accuracy_list = trainer.train()
    metric_list.append(valid_accuracy_list[-1])
    hyperparam['val_accuracy'] = valid_accuracy_list[-1]
    hyperparam['train_accuracy'] = train_accuracy_list[-1]

idx = np.argsort(metric_list)
hyperparameter_sorted = np.array(hyperparameter_try)[idx].tolist()
df = pd.DataFrame.from_dict(hyperparameter_sorted)

Epoch 1/100: 100%|██████████| 14/14 [00:29<00:00,  2.10s/it]


Epoch 1: Train Loss: 2.4814, Train Acc: 16.5389, Val Loss: 2.4420, Val Acc: 22.5000
Epoch 0 - Keeping weights


Epoch 2/100: 100%|██████████| 14/14 [00:27<00:00,  1.96s/it]


Epoch 2: Train Loss: 2.3668, Train Acc: 22.2309, Val Loss: 2.3894, Val Acc: 23.4375


Epoch 3/100: 100%|██████████| 14/14 [00:27<00:00,  1.97s/it]


...

Epoch 22: Train Loss: 1.8920, Train Acc: 39.0628, Val Loss: 2.0730, Val Acc: 40.3646
Epoch 21 - Best metrics: 40.364583333333336 - Keeping weights


Epoch 23/100: 100%|██████████| 7/7 [00:41<00:00,  5.94s/it]


Epoch 23: Train Loss: 1.8644, Train Acc: 40.1068, Val Loss: 2.0450, Val Acc: 40.1042
Epoch 22 - Metrics did not improve, wait: 1


Epoch 24/100:  71%|███████▏  | 5/7 [00:36<00:14,  7.33s/it]


KeyboardInterrupt: 

In [ ]:
from tabulate import tabulate

idx = np.argsort(metric_list)
hyperparameter_sorted = np.array(hyperparameter_try)[idx].tolist()
df = pd.DataFrame.from_dict(hyperparameter_sorted)

print(tabulate(df, headers='keys', tablefmt='psql'))

+----+----------------+------------------+------------------+---------------+--------------+------------+------------+-------------+-----------------+--------------------+--------------------+-------------+----------------------+-----------+------------------+----------------+--------------+-----------------------+--------------+----------------+------------------+
|    |   weight_decay | stride_pool      | stride_conv      |   patience_lr |   output_dim | nesterov   |   momentum |   max_epoch |   learning_rate | kernel_size_pool   | kernel_size_conv   |   input_dim |   hidden_layers_size | filters   | early_stopping   |   dropout_rate |   batch_size | batch_normalization   | activation   |   val_accuracy |   train_accuracy |
|----+----------------+------------------+------------------+---------------+--------------+------------+------------+-------------+-----------------+--------------------+--------------------+-------------+----------------------+-----------+------------------+----

: 

In [ ]:
hyperparameter_final = dict(
    input_dim=input_dim,
    output_dim=n_classes,
    hidden_layers_size=5000,
    activation='relu',
    kernel_size_conv=[(57, 6), (1, 3)],
    kernel_size_pool=[(4, 3), (1, 3)],
    stride_conv=[(1, 1), (1, 1)],
    stride_pool=[(1, 3), (1, 3)],
    filters=[80, 80],
    batch_normalization=False,
    dropout_rate=0.5,
    # trainer hyperparameters
    learning_rate=0.005,
    weight_decay=0.001,
    momentum=0.85,
    nesterov=True,

    # questionable hyperparameters
    #batch_size=batch_size,
    max_epoch=100,

    #Early stopping and sceduler
    patience_lr=10,
    # here it said was false, but??
    early_stopping=True,
    batch_size=64
)

In [25]:
from ClassesML.AudioModel import AudioModel
model_final = AudioModel(hyperparameter_final).to(device)

In [26]:
import torch.nn.functional as F
def predict_clip(model, segment_tensors, device, method='prob'):
    """
    Generate prediction for a whole clip given its segments.
    
    segment_tensors: tensor of shape (n_segments, channels, mel_bands, frames)
    method: 'majority' or 'prob'
    """
    model.eval()
    segment_tensors = segment_tensors.to(device)
    with torch.no_grad():
        outputs = model(segment_tensors)  # (n_segments, num_classes)
        probs = F.softmax(outputs, dim=1)

    if method == 'majority':
        preds = torch.argmax(probs, dim=1)
        counts = torch.bincount(preds)
        clip_pred = torch.argmax(counts).item()
    elif method == 'prob':
        avg_probs = probs.mean(dim=0)
        clip_pred = torch.argmax(avg_probs).item()
    else:
        raise ValueError("method must be 'majority' or 'prob'")

    return clip_pred


In [27]:
from ClassesML.AudioTrainer import AudioTrainer
trainer_final = AudioTrainer(model_final, train_loader, val_loader, hyperparameter_final, device=device)
trainer_final.train()

Epoch 1/2:   0%|          | 0/19 [00:00<?, ?it/s]

Epoch 1/2: 100%|██████████| 19/19 [02:10<00:00,  6.87s/it]


Epoch 1: Train Loss: 2.4909, Train Acc: 17.2626, Val Loss: 2.4540, Val Acc: 33.2366


Epoch 2/2: 100%|██████████| 19/19 [02:34<00:00,  8.13s/it]


Epoch 2: Train Loss: 2.3631, Train Acc: 29.9610, Val Loss: 2.3919, Val Acc: 33.1830


([17.26258581235698, 29.961026887871856],
 [33.236607142857146, 33.183035714285715])

### Evaluate the model using the 5-fold cross validation

In [28]:
import warnings
def extract_segments_with_deltas(file_path, variant='short', silence_threshold=-80):
    """
    Extracts 2-channel (log-mel + delta) spectrogram segments from an audio file.

    Parameters:
    - file_path (str): Path to audio file.
    - variant (str): 'short' (41 frames, 50% overlap) or 'long' (101 frames, 90% overlap).
    - silence_threshold (float): dB threshold for discarding low-energy segments.

    Returns:
    - np.ndarray: Array of shape (n_segments, 2, 60, frames_per_segment)
    """
    # Config
    sr = 22050
    n_fft = 1024
    hop_length = 512
    n_mels = 60

    if variant == 'short':
        frames_per_segment = 41
        overlap = 0.5
    elif variant == 'long':
        frames_per_segment = 101
        overlap = 0.9
    else:
        raise ValueError("variant must be 'short' or 'long'")

    try:
        # Load audio in mono
        y, _ = librosa.load(file_path, sr=sr, mono=True)

        # Skip empty or very short files
        if len(y) < n_fft:
            warnings.warn(f"File too short to process: {file_path}")
            return np.empty((0, 2, 60, frames_per_segment))

        # Compute log-mel spectrogram
        mel_spec = librosa.feature.melspectrogram(
            y=y,
            sr=sr,
            n_fft=n_fft,
            hop_length=hop_length,
            n_mels=n_mels
        )
        log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)

        # Normalize safely
        mean = np.mean(log_mel_spec)
        std = np.std(log_mel_spec)
        if std == 0:
            warnings.warn(f"Zero std encountered in file: {file_path}")
            return np.empty((0, 2, 60, frames_per_segment))

        log_mel_spec = (log_mel_spec - mean) / std

        # Compute deltas
        delta_spec = librosa.feature.delta(log_mel_spec)

        # Segmenting
        step = int(frames_per_segment * (1 - overlap))
        segments = []

        for start in range(0, log_mel_spec.shape[1] - frames_per_segment + 1, step):
            seg = log_mel_spec[:, start:start + frames_per_segment]
            delta = delta_spec[:, start:start + frames_per_segment]

            # Skip silent segments
            if np.mean(seg) < silence_threshold:
                continue

            stacked = np.stack([seg, delta], axis=0)
            segments.append(stacked)

        return np.stack(segments) if segments else np.empty((0, 2, 60, frames_per_segment))

    except Exception as e:
        warnings.warn(f"Failed to process {file_path}: {e}")
        return np.empty((0, 2, 60, frames_per_segment))

In [18]:
import pandas as pd
from sklearn.model_selection import KFold

# Load your dataset
df = pd.read_csv('data/Animal_Sound_processed.csv')  # Update path if needed

# Prepare 5-Fold cross-validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracy_list = []
val_loader_list = []
train_accuracy_per_epoch_list = []
val_accuracy_per_epoch_list = []




for fold, (train_val_idx, val_idx) in enumerate(kf.split(df)):
    df_train_val = df.iloc[train_val_idx].reset_index(drop=True)
    df_test = df.iloc[val_idx].reset_index(drop=True)

    print(f"Fold {fold + 1}")
    print(f"Train/Val samples: {len(df_train_val)}")
    print(f"Test samples: {len(df_test)}")

    # Create datasets for this fold
    dataset_train = AnimalSoundDataset(df_train_val, split='train', split_ratio=0.75, seed=42)
    dataset_val = AnimalSoundDataset(df_train_val, split='val', split_ratio=0.75, seed=42)
    train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=False)
    val_loader_list.append(val_loader)

    # Initialize model and trainer
    model_final = AudioModel(hyperparameter_final).to(device)
    trainer_final = AudioTrainer(model, train_loader, val_loader, hyperparameter_final, device=device)
    train_acc_list, val_acc_list = trainer_final.train()
    train_accuracy_per_epoch_list.append(train_acc_list)
    val_accuracy_per_epoch_list.append(val_acc_list)
    # Save the model for this fold
    model_save_path = f'model_fold_{fold + 1}.pth'
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}\n")

    # Evaluate on test set
    total_predictions = 0
    correct_predictions = 0
    for path, label in zip(df_test['path'], df_test['name']):

        test_segments = extract_segments_with_deltas(path, variant='short')
        predicted_sound = predict_clip(model, torch.tensor(test_segments, dtype=torch.float32).to(device), device, method='prob')
        

        predicted_label = dataset_train.classes[predicted_sound]
        
        if predicted_label == label:
            correct_predictions += 1
        total_predictions += 1
    accuracy = correct_predictions / total_predictions * 100 if total_predictions > 0 else 0
    accuracy_list.append(accuracy)
    print(f"Fold {fold + 1} - Test Accuracy: {accuracy:.4f}\n")


        

   

Fold 1
Train/Val samples: 325
Test samples: 325
Classes found: ['bear', 'cat', 'chicken', 'cow', 'dog', 'dolphin', 'donkey', 'elephant', 'frog', 'horse', 'lion', 'monkey', 'sheep']
Total segments extracted: 1442
train set contains 1081 segments.
Classes found: ['bear', 'cat', 'chicken', 'cow', 'dog', 'dolphin', 'donkey', 'elephant', 'frog', 'horse', 'lion', 'monkey', 'sheep']
Total segments extracted: 1442
val set contains 361 segments.


Epoch 1/2: 100%|██████████| 9/9 [00:59<00:00,  6.56s/it]


Epoch 1: Train Loss: 1.7755, Train Acc: 43.9998, Val Loss: 1.9040, Val Acc: 48.0184


Epoch 2/2: 100%|██████████| 9/9 [00:45<00:00,  5.05s/it]


Epoch 2: Train Loss: 1.6610, Train Acc: 46.9740, Val Loss: 1.8662, Val Acc: 44.1443
Model saved to model_fold_1.pth

Fold 1 - Test Accuracy: 23.0769

Fold 2
Train/Val samples: 325
Test samples: 325
Classes found: ['bear', 'cat', 'chicken', 'cow', 'dog', 'dolphin', 'donkey', 'elephant', 'frog', 'horse', 'lion', 'monkey', 'sheep']
Total segments extracted: 1467
train set contains 1100 segments.
Classes found: ['bear', 'cat', 'chicken', 'cow', 'dog', 'dolphin', 'donkey', 'elephant', 'frog', 'horse', 'lion', 'monkey', 'sheep']
Total segments extracted: 1467
val set contains 367 segments.


...

Total segments extracted: 2301
val set contains 576 segments.


Epoch 1/2: 100%|██████████| 14/14 [00:59<00:00,  4.22s/it]


Epoch 1: Train Loss: 2.2818, Train Acc: 30.2611, Val Loss: 2.3417, Val Acc: 28.4375
Epoch 0 - Keeping weights


Epoch 2/2:   0%|          | 0/14 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [53]:
max_accuracy_idx = accuracy_list.index(max(accuracy_list))
max_val_loader = val_loader_list[max_accuracy_idx]
max_train_acc_list = train_accuracy_per_epoch_list[max_accuracy_idx]
max_val_acc_list = val_accuracy_per_epoch_list[max_accuracy_idx]

In [54]:
from sklearn.metrics import confusion_matrix
from Utilities.Utilities import Utilities
import seaborn as sns

def plot_confusion_matrix_fashion(y, y_hat):
    accuracy = Utilities.compute_accuracy(y, y_hat)

    y_hat = np.argmax(y_hat, 1)

    cm = confusion_matrix(y, y_hat)
    cm_normalized = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

    label_map = {0: 'bear', 1: 'cat', 2: 'chicken',
                    3: 'cow', 4: 'dog', 5: 'dolphin',
                    6: 'donkey', 7: 'elephant', 8: 'frog', 9: 'horse',
                    10: 'lion', 11: 'monkey', 12: 'sheep'}
    
    plt.figure()
    plt.subplot(1, 1, 1)
    sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap="Blues",
                xticklabels=[label_map[i] for i in range(13)],
                yticklabels=[label_map[i] for i in range(13)])
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.title("Confusion matrix - Accuracy: " + str(accuracy))
    plt.tight_layout()
    return plt.show()

x_list = []
y_list = []

for x_batch, y_batch in max_val_loader:
    x_list.append(x_batch)
    y_list.append(y_batch)

x_all = torch.cat(x_list).to(device)
y_all = torch.cat(y_list).to(device)

# === Run model and evaluate ===
with torch.no_grad():
    y_hat = model(x_all)

y_true = y_all.cpu().numpy()
y_pred_logits = y_hat.cpu().numpy()

# === Plot confusion matrix ===
plot_confusion_matrix_fashion(y_true, y_pred_logits)

In [55]:
plt.figure()
plt.plot(max_train_acc_list, 'b', label='Train accuracy')
plt.plot(max_val_acc_list, 'r', label='Valid accuracy')
plt.legend()
plt.show()